# Advection of a Cosine Wave

Let's say we have a non-propagating wave which, at $t=0$, is given by

$$ \psi(x,t=0) = \cos \Big( \frac{2 \pi x}{1000 \text{ m}} \Big) .$$

So we have a wave with $\lambda = 1000$ meters, shown below.

In [ ]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt

# Define parameters
L = 1000.

# Define x grid
x = np.linspace(0,2000,100)

# Define wave function
psi = np.cos(2 * np.pi * x / L)

# Plot the function
fig,ax = plt.subplots(figsize=(12,6));
ax.plot(x,psi);
ax.set_xlabel("$x$",fontsize=14);
ax.set_ylabel("$\psi$",fontsize=14);
ax.set_xlim(0,2000);

Now, let's say this wave resides in a flow of constant velocity, $u$. The wave here will be advected by the flow and will evolve as such:

$$ \displaystyle \frac{\partial \psi}{\partial t} = - u \frac{\partial \psi}{\partial x} .$$

We can used finite differences to perform the partial derivatives and solve for $\psi (t,x)$. If we let $n$ denote indexing in time and $i$ denote indexing in the $x$ direction, we can rewrite the above with finite difference as 

$$ \frac{\psi^{n+1} - \psi^{n}}{\Delta t} = - u \frac{\psi_{i} - \psi_{i-1}}{\Delta x} .$$

The notation I will use is to superscript the time indices and the subscript the space indices. The parentheses are used to differentiate that we are not saying "to the $n$th power". Here, $\Delta t$ is size of the time step between two time indices, $t^{n+1} - t^{n}$, and similarly, $\Delta x$ is the grid spacing used on the $x$ grid, $x_{i+1} - x_{i}$. Also, note the two finite difference schemes. We've applied forward differencing in time and backwards differencing in space. To update $\psi^{n+1}$, we have to then solve for it using the above equation.

$$ 
\frac{\psi^{(n+1)} - \psi^{(n)}}{\Delta t} = - u \frac{\psi_{i} - \psi_{i-1}}{\Delta x} \\
\psi^{(n+1)} - \psi^{(n)} = \Delta t \Big( - u \frac{\psi_{i} - \psi_{i-1}}{\Delta x} \Big) \\
\psi^{(n+1)} = \psi^{(n)} - u \Delta t \frac{\psi_{i} - \psi_{i-1}}{\Delta x} 
$$

There is still one big problem. Because of the differencing scheme we are using, when we try to use the numerical solution at $i=0$, we have an undefined $\psi_{i-1}$. To get around this, we have to assume some type of boundary condition. Since we have a periodic function, and it is existing within one of the normal modes of the domain, it's natural to assume a periodic boundary condition, where the wave could exist on one side of the domain and return on the opposite side. To do this, at $i=0$ we simply set

$$ \psi^{(n+1)}_0 = \psi^{(n+1)}_{n_x-1} $$

where $n_x$ is the number of points in $x$.

Let's see how this might look for a flow with constant velocity $u=10$ m s$^{-1}$.

In [ ]:
# Set up parameters
u = 10.
L = 1000.
nx = 21
nt = 10
dx = 100.
dt = 10.

# Set up grids
x = np.arange(0,nx*dx,dx)
t = np.arange(0,nt*dt,dt)

# Create the initial function at t=0
psi_present = np.cos(2.*np.pi*x/L)

# Plot function
fig,ax = plt.subplots(figsize=(12,6));
ax.plot(x,psi_present);
ax.set_xlabel("$x$",fontsize=14);
ax.set_ylabel("$\psi$",fontsize=14);
ax.set_title(f"t = {0.}")
ax.set_xlim(0,2000)

# Loop through at times and predict new wave function
psi_future = np.copy(psi_present)
for n in range(0,nt):
    
    # Reset the present wave function
    psi_present = np.copy(psi_future)
    
    # Update according to advection equation
    for i in range(1,nx):
        psi_future[i] = psi_present[i] - (u*dt*(psi_present[i]-psi_present[i-1])/dx)
    
    # Apply periodic boundary condition
    psi_future[0] = psi_future[nx-1]
    
    # Plot function
    fig,ax = plt.subplots(figsize=(12,6));
    ax.plot(x,psi_future);
    ax.set_xlabel("$x$",fontsize=14);
    ax.set_ylabel("$\psi$",fontsize=14);
    ax.set_title(f"t = {(n+1)*dt}")
    ax.set_xlim(0,2000)

So now we have a wave moving as a result of the background flow rather than the wave's internal propagation itself. We used some for-loops to perform this update, but ideally we want to vectorize this. Let's try that instead. We'll vectorize the backwards difference used to calculate the zonal gradient of the wave for each time step, leaving us with just one loop for the forward differencing in time. We have to keep this one, because we do not have a pre-defined $n+1$ on which to vectorize the implementation since that's what we are solving for.

In [ ]:
# Set up parameters
u = 10.
L = 1000.
nx = 21
nt = 10
dx = 100.
dt = 10.

# Set up grids
x = np.arange(0,nx*dx,dx)
t = np.arange(0,nt*dt,dt)

# Create the initial function at t=0
psi_present = np.cos(2.*np.pi*x/L)

# Plot function
fig,ax = plt.subplots(figsize=(12,6));
ax.plot(x,psi_present);
ax.set_xlabel("$x$",fontsize=14);
ax.set_ylabel("$\psi$",fontsize=14);
ax.set_title(f"t = {0.}")
ax.set_xlim(0,2000)

# Loop through at times and predict new wave function
psi_future = np.copy(psi_present)
for n in range(0,nt):
    
    # Reset the present wave function
    psi_present = np.copy(psi_future)
    
    # Update according to advection equation
    psi_future[1:nx] = psi_present[1:nx] - (u*dt*(psi_present[1:nx]-psi_present[0:nx-1])/dx)
    
    # Apply periodic boundary condition
    psi_future[0] = psi_future[nx-1]
    
    # Plot function
    fig,ax = plt.subplots(figsize=(12,6));
    ax.plot(x,psi_future);
    ax.set_xlabel("$x$",fontsize=14);
    ax.set_ylabel("$\psi$",fontsize=14);
    ax.set_title(f"t = {(n+1)*dt}")
    ax.set_xlim(0,2000)